# Exercises

## Exercise

If $S_t$ satisfies geometric Brownian motion and $S_0>0$, prove that $S_t>0$ at all times $t \geq 0$.

It has been proved using Ito's Lemma that 
$$
S_t = S_0 \exp( (\mu - \frac{1}{2} \sigma^2)t + \sigma W_t).
$$
Since the exponential of a real number is positive and since $S_0$ is positive, $S_t>0$.

## Exercise

Write down the Euler scheme for the equation
$$
d S_t = S_t (\mu \, dt + \sigma \, d W_t)
$$
on a grid of size $\delta t$. Suppose that $S_0>0$. Prove that there is a non-zero probability that $S_t<0$ at any time $t>0$ in the grid.

The Euler scheme is
$$
S_{t+\delta t} = S_t + S_t( \mu \delta t + \sigma \sqrt{t} \epsilon_t)
$$
for a standard normally distributed random variable $\epsilon$. Therefore
$$
S_{t+\delta t} = S_t \, n_t
$$
where $n_t$ follows a (non-standard) normal distribution. So long as $S_t\neq 0$ there is a non-zero probability of $n_t$ being
either positice or negative, and hence there is a non-zero probability of $S_{t+\delta t}$ being negative, conditioned on the sign of $S_t$. The probability that $n_t=0$ is zero. Hence by induction, the unconditional probability that $S_t=0$ is zero and the unconditional probability that $S_t<0$ is non-zero.

